### Changelog
#### 1. 使用原始数据，未做normalize，使用2层same padding(k_size=3,padding=1)卷积，步长为2的池化，两个全连接层，88.7%的准确率(Linux platform)
#### 2. 和1中的所有操作相同的，但是做了normalize，准确率88.84%(linux platform)
#### 3. 利用2中的regularization，将卷积核的大小改为5，padding为2，准确率为89.15%(windows platform)
#### 4. 在3的基础上，更换optimizer算法由SGD变为Adam，准确率为89.59%(Linux platform)
#### 5. 在2的基础上，更换optimizer算法由SGD变为Adam，准确率为89.9%(Linux platform)
#### 6. 加入net.eval(),准确率上升到90.94%(Linux platform)
#### 7. 加入一个多加入了一个全连接隐层，准确率上升到91.75%(Linux platform)
#### 8. 全连接层隐层神经元比例调整为9:3:2之后，同时加入一个新的卷基层，准确率为91.89%(Linux platform)
#### 9. 去掉第一个卷基层后的BN层，调整第三个卷基层的感受野为5x5，准确率为92.07%(Linux platform)
#### 10. 提高感受野，准确率为92.28%(Linux platform)
#### 11. 增加第二，第三卷积层的输出feature map通道（channel）数，准确率为92.49%(Linux platform)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
label_to_description = {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress',
                        4: 'Coat', 5: 'Sandal', 6: 'Shirt',
                       7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot',
                        }

In [ ]:
path_to_datasets = '/home/lor/Datasets/FashionMNIST/FashionMNIST/processed/training.pt'
# path_to_datasets = 'E:/data/FashionMNIST/FashionMNIST/processed/training.pt'
samples = torch.load(path_to_datasets)
features = samples[0]
targets = samples[1]
print(features.shape, targets.shape)
print(targets.max())
print(targets.min())

In [ ]:
ax = plt.subplot(2, 2, 1)
plt.imshow(np.array(features[3]), cmap='gray')
ax.set_title(label_to_description[targets.storage()[3]])
ax = plt.subplot(2, 2, 2)
plt.imshow(np.array(features[4]), cmap='gray')
ax.set_title(label_to_description[targets.storage()[4]])
plt.show()
# plt.title(label_to_description[targets.storage()[3]])
# label_to_description[targets.storage()[3]]

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class UserDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        target = self.targets[idx]
        feature = self.features[idx].unsqueeze(0).float()
        feature = (feature - torch.min(feature)) / (torch.max(feature) - torch.min(feature))
        return (target, feature)

In [ ]:
train_dataset = UserDataset(features, targets)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
one_iter = next(iter(train_dataloader))
one_iter[0].reshape(-1)

In [ ]:
for i in range(10):
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(one_iter[1][i][0], cmap='gray')
    plt.axis('off')
    ax.set_title(label_to_description[one_iter[0].storage()[i]])

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Sequential(
                        nn.Conv2d(1, 8, kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(8),
                        nn.ReLU(),
                        nn.Conv2d(8, 18, kernel_size=(5, 5), stride=1, padding=2),
                        nn.BatchNorm2d(18),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        
        self.conv2 = nn.Sequential(
                        nn.Conv2d(18, 26, kernel_size=(5, 5), stride=1, padding=2),
                        nn.BatchNorm2d(26),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        
        self.fc1 = nn.Linear(7 * 7 * 26, 424)
        self.fc2 = nn.Linear(424, 141)
        self.fc3 = nn.Linear(141, 10)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        out = self.fc2(F.relu(out))
        res = self.fc3(F.relu(out))
        
        return res
    
def weight_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        nn.init.zeros_(m.bias)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Network()
# net.apply(weight_init)
net.to(device)
loss = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=1e-3)
optimizer = optim.Adam(net.parameters(), lr=1e-3)

total_step = len(train_dataloader)

In [ ]:
num_epochs = 10
net.train()
for epoch in range(num_epochs):
    for i, (target, feature) in enumerate(train_dataloader):
        target, feature = target.to(device), feature.to(device)
        out = net(feature)
#         out = out.reshape(-1)
        target = target.reshape(-1)
        criterion = loss(out, target)
        
        optimizer.zero_grad()
        criterion.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, criterion.item()))
        

In [ ]:
### 验证
# test_path = 'E:/data/FashionMNIST/FashionMNIST/processed/test.pt'
test_path = '/home/lor/Datasets/FashionMNIST/FashionMNIST/processed/test.pt'
test_dataset = torch.load(test_path)
print(test_dataset[0].shape)
print(test_dataset[1].shape)

In [ ]:
test_dataset = UserDataset(test_dataset[0], test_dataset[1])
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=4)

In [ ]:
total = 10000
pred_count = 0
net.eval()
for (target, feature) in test_dataloader:
#     print(target.shape)
#     print(feature.shape)
    target = target.to(device)
    feature = feature.to(device)
    out = net(feature)
#     print(torch.argmax(out, 1))
#     print(target)
#     print(torch.argmax(out, 1) == target)
#     print((torch.argmax(out, 1) == target).sum().item())
    pred_count = pred_count + (torch.argmax(out, 1) == target).sum().item()
#     break
print("Accuracy: {0}%".format(100 * pred_count / total))

In [ ]:
# Save the model
torch.save(net.state_dict(), './net.pt')

In [ ]:
# Load the model
oth_net = Network()
oth_net.load_state_dict(torch.load('./net.pt'))

In [ ]:
oth_net.eval()